In [ ]:
! mkdir -p datasets
%cd datasets
! wget -nc https://raw.githubusercontent.com/pablonoya/zigzag-ml/master/datasets/Iris.csv
%cd ..

# Redes Neuronales Artificiales
Quiero empezar con una cita del libro *Hands-On Machine Learning with Scikit-Learn, Keras and Tensorflow*
> Los pájaros nos inspiraron para volar, la Bardana inspiró el Velcro, y la naturaleza ha inspirado innumerables inventos más. Parece lógico, pues, fijarnos en la arquitectura del cerebro para inspirarnos en la construcción de una máquina inteligente.
>
> Aurélien Géron, 2019, p. 310

Estas redes artificiales son a menudo representadas por **capas de neuronas totalmente conectadas entre sí**, podemos distinguir tres partes:
1. Capa de entrada
2. Una o varias capas ocultas
3. Capa de salida.

![Red neuronal artificial](./img/10.1_Artificial_neural_network.png)

## Implementación con Keras
Este tipo de redes son denominadas *fully connected* y pueden implementarse como un modelo `Sequential` con capas `Dense`, porque son una secuencia de capas densamente conectadas 😉.  
Tomaremos ambos elementos del módulo `keras` de tensorflow 2, que tiene submódulos como `models` y `layers`

In [ ]:
# modelo secuencial
from tensorflow.keras.models import Sequential
# capa densa
from tensorflow.keras.layers import Dense

Podemos definir las capas del modelo pasándolas en una lista al momento de instanciarlo, cada capa `Dense` conecta las neuronas por nosotros, sólo debemos definir el número de neuronas en la primera, correspondiente a la capa de entrada, con el argumento `input_shape` el cual recibe una tupla, pero dejaremos la segunda dimensión vacía.  
Para las capas posteriores, no es necesario definir este argumento, es calculado a partir de la capa anterior 😉

In [ ]:
model = Sequential([
    Dense(3, input_shape=(2,)),
    Dense(2)
])

Podemos ver un resumen de la arquitectura de nuestra red con el método `summary` este nos muestra el tipo de capas, las shapes de salida, donde None significa un número variable, y el número de parámetros.

In [ ]:
model.summary()

También podemos instanciar nuestro modelo para añadir después las capas con el método `add`

In [ ]:
model2 = Sequential()
model2.add(Dense(3, input_shape=(2,)))
model2.add(Dense(2, input_shape=(2,)))

Obteniendo el mismo resultado

In [ ]:
model2.summary()

# Neurona biológica, neurona artificial
Las **neuronas artificiales** nacieron de la idea de **replicar las neuronas biológicas**, estas **reciben un impulso por medio de sus dendritas y lo transmiten a través de su axón** hacia otras neuronas, formando una red.

![Neurona biológica](img/10.2.1_Biological_neuron.png)

De la misma manera, una neurona artificial **recibe la información de otras neurona asignándoles pesos** o _weights_, realiza una **sumatoria de las entradas ponderadas**, y **pasa el resultado por una función de activación** para propagar una respuesta.

![Neurona artificial](img/10.2.2_Artificial_neuron.png)

¿Recuerdas el término independiente?, también conocido como *bias* y también presente en las redes neuronales cumple la misma función de dotar de mayor libertad al ajuste de parámetros.

# Función de activación
Cumple la tarea de romper la linealidad de las redes neuronales, podemos usar funciones como la sigmoide, pero la más utilizada es la función *Rectified Linear Unit* o **ReLU** para los amigos. Se define con una fórmula muy sencilla, la cual implementaremos.

$$R(z) = max(0, z)$$

In [ ]:
import numpy as np

def relu(z):
    return np.max((0, z))

In [ ]:
import matplotlib.pyplot as plt

xpoints = np.linspace(-1, 1, 11)
ypoints = [relu(x) for x in xpoints]

plt.plot(xpoints, ypoints)
plt.grid()
plt.ylim(-0.2, 1.2)

**Esta función se aplica entre capas**, a excepción de la última, cuya función de activación cambiará dependiendo de nuestra tarea:

- Función **linear**, equivalente a no usar ninguna función, si deseamos realizar **regresión**.
- Función **sigmoide** si deseamos realizar **clasificación binaria**.
- Función **softmax** para **clasificación múltiple**.

Además el **número de neuronas en la capa de salida** dependerá de la tarea que busquemos resolver, siendo **sólo una para regresión o clasificación binaria** y el **número de clases** para la **clasificación múltiple**.

# Clasificando Iris
Veamos otra vez más nuestro dataset de Iris

In [ ]:
import pandas as pd

data = pd.read_csv('./datasets/Iris.csv')
data.drop('Id', axis='columns', inplace=True)

data.head()

Ahora dividimos en features y labels, luego en conjuntos train y test

In [ ]:
from sklearn.model_selection import train_test_split

X = data.drop('Species', 'columns')

# utilizaremos one-hot encoding
y = pd.get_dummies(data['Species'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
len(X_train)

## Modelo: Red neuronal
Tenemos 4 features, ese será el número de neuronas en nuestra capa de entrada.  
Para las capas ocultas, usualmente se recomienda usar potencias de 2, además de seguir con la forma que vimos al principio lara que la red sea más grande en el centro, definamos 2 capas de 8 neuronas.

In [ ]:
model_iris = Sequential([
    Dense(8, input_dim=4),
    Dense(8, activation='relu'),
    # capa de salida, activación softmax
    Dense(3, activation='softmax')
])

## Configuración
Mediante el método `compile` podemos definir opciones adicionales, **la función de costo es obligatoria**, definida por el parámetro `loss` siendo `"categorical_crossentropy"` para el caso de softmax.  

También cambiaremos el **optimizador** por el [Descenso del gradiente estocástico](Descenso del gradiente estocástico), el cual intercambia la precisión del gradiente por la velocidad, pues calcula el gradiente a partir de una muestra aleatoria de datos en lugar de utilizar todos, esta idea es bastante útil en datasets gigantes 🤯. Podemos establecerlo con el parámetro `optimizer="sgd"`.  

Además podemos incluir métricas adicionales, el costo se nos mostrará por defecto, incluyamos la exactitud o `"acc"` dentro de una lista 😄

In [ ]:
model_iris.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=['acc'])

## Entrenamiento
También contamos con el método `fit` al igual que sklearn, pero debemos indicarle el número máximo de iteraciones o `epochs` y opcionalmente el `batch_size`, que indica el **tamaño de la muestra aleatoria** que usará para actualizar el gradiente.

In [ ]:
model_iris.fit(X_train, y_train, epochs=100, batch_size=20)

Para cada *epoch* tenemos 6 *batches*, $6 \times 20 = 120$, el número de datos de entrenamiento.

## Evaluación
El modelo tiene un método `evaluate` que puede evaluar según las métricas que definimos al configurar

In [ ]:
# predicción y evaluación
y_hat = model_iris.predict(X_test)
model_iris.evaluate(X_test, y_hat)

# Deep Learning
Cuando hablamos de *Deep Learning* o Aprendizaje profundo nos referimos al uso de redes neuronales para resolver tareas más complejas, estamos ante una **especialización** del *Machine Learning*

![Deep learning dentro de ML](img/10.5_IA_ML_DL.png)

Tareas como dotar a las máquinas la capacidad de **reconocer dígitos escritos a mano** ✍️🤯

# Reconociendo dígitos
MNIST es un dataset con una gran cantidad de dígitos escritos a mano, son 70 mil 😱 en forma de imágenes de 28 x 28 pixeles.  
Es toda una leyenda, por lo que muchas veces se utiliza como hola mundo 😄 y podemos cargarlo gracias a keras

In [ ]:
from tensorflow.keras.datasets import mnist

La versión de keras ya cuenta con los conjuntos de entrenamiento y prueba, devueltos por la función `load_data` con una estructura un poco extraña para desempaquetar 🤔

In [ ]:
train, test = mnist.load_data()

X_train, y_train = train
X_test, y_test = test

Veamos el primer ejemplo de entrenamiento con `imshow` de matplotlib, el argumento `cmap="Greys"` nos permite ver la imagen en escala de grises

In [ ]:
plt.imshow(X_train[0], cmap="Greys")

Este es el número...

In [ ]:
y_train[0]

Además, ya tenemos los datos como deberíamos, en números 🔢

In [ ]:
print(X_train[0])

## Preprocesamiento
Tenemos **valores de 0 a 255**, los valores más altos son grises más oscuros. Pero **esta diferencia de valores puede dar problemas** con una red neuronal, por lo que un preprocesamiento común es **dividirlos por 255** para dejarlos en un **rango de 0 a 1**.

In [ ]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

Los labels están en un sólo número entero, pero deberían estar en One-hot encoding.  
Por suerte to_categorical() de keras puede ayudarnos con eso 😉

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train_cat = to_categorical(y_train)
y_train_cat

## Arquitectura de la red
Es bastante sencillo definir la capa de salida, tenemos dígitos del 0 a 9 y nuestra tarea es clasificación multiclase, tendremos una **capa de salida con 10 neuronas y la activación softmax**.

La entrada es una **matriz de 28 x 28**, esto son 784 números, usaremos ese número de neuronas en la capa de entrada para esta red, pero antes debemos "aplanar" esta matriz para convertirla en un sólo vector de 784 números.

![aplanar matriz](https://i.imgur.com/dDYphPB.png)

Podemos lograr esto usando una capa `Flatten` de keras, que recibirá un input shape de (28, 28).  
Y para las **capas ocultas** puedes usar 1024 o 512 unidades 😃

In [ ]:
from tensorflow.keras.layers import Flatten

model_mnist = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(___, activation='relu'),
    Dense(___, activation='relu'),
    Dense(10, activation='softmax')
])

## Entrenamiento
Usaremos las mismas opciones que en el caso de las Iris, cambiando el **batch size** a 128, también **se recomienda usar potencias de 2** para este argumento. En el caso anterior elegí 20 para dividir exactamente los 120 ejemplos de entrenamiento 😎.

In [ ]:
model_mnist.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=['acc'])

model_mnist.fit(X_train_scaled, y_train_cat, epochs=3, batch_size=128)

In [ ]:
y_hat = model_mnist.predict(X_test_scaled)
model_mnist.evaluate(X_test_scaled, y_hat)

# Ejercicios
Muestra un dígito y del conjunto de prueba junto a la predicción de la red neuronal 😉.

El número de capas ocultas, el número de neuronas por capa y el batch size son **hiperparámetros** trata de ajustarlos para mejorar el rendimiento de la red que reconoce dígitos del mnist 😎.

# Epílogo
El *Deep Learning* es una de las áreas que más crecimiento ha tenido en los últimos años, posibilita que las máquinas sean **capaces de aprender tareas mucho más complejas, sobre imágenes, texto, audio e incluso video** 🤯

Tales cuestiones requieren de un amplio estudio, y de seguro estás con muchas ganas de aprender 😃.

Esta colección de notebooks nació del deseo de compartir las bases del *Machine Learning*, sobre las que puedes apoyarte siempre que lo requieras, mi mayor deseo es haberte transmitido el entusiasmo que tengo por esta área y compartir un poco el cómo funciona 🤔.

¡Gracias por quedarte hasta el final! 😊  
Te deseo un feliz viaje por el mundo del Machine Learning 😃.